In [18]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd 
import numpy as np 

In [19]:
# 1. Gather Data
boston_dataset = load_boston()
data = pd.DataFrame(data=boston_dataset.data, columns=boston_dataset.feature_names)
features = data.drop(['INDUS', 'AGE'], axis=1)
log_prices = np.log(boston_dataset.target)
target = pd.DataFrame(log_prices, columns=['PRICE'])

In [20]:
features

,CRIM,ZN,CHAS,NOX,RM,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,0.0,0.538,6.575,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,0.0,0.469,6.421,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,0.0,0.469,7.185,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,0.0,0.458,6.998,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,0.0,0.458,7.147,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,0.0,0.573,6.593,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,0.0,0.573,6.120,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,0.0,0.573,6.976,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,0.0,0.573,6.794,2.3889,1.0,273.0,21.0,393.45,6.48


In [21]:
CRIME_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8
property_stats = feat_values = features.mean().values.reshape((1,11))

In [22]:
regr = LinearRegression().fit(features, target)
fitted_vals = regr.predict(features)

# Challenge: Calculate the MSE & RMSE
MSE = mean_squared_error(target, fitted_vals)
RMSE = np.sqrt(MSE)

In [23]:
def get_log_estimate(nr_rooms, students_per_classroom, 
                    next_to_river=False, high_confidence=True):
    
    # Configure Property
    property_stats[0][RM_IDX] = nr_rooms
    property_stats[0][PTRATIO_IDX] = students_per_classroom
    
    if next_to_river:
        property_stats[0][CHAS_IDX] = 1
    else: 
        property_stats[0][CHAS_IDX] = 0
         
    # Make Prediction
    log_estimate = regr.predict(property_stats)[0][0]
    
    # Calc Range
    if high_confidence:
        # 95% Interval Range
        upper_bound = log_estimate + 2*RMSE
        lower_bound = log_estimate - 2*RMSE
        interval = 95
    else:
        # 68% Interval Range
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 68
        
    return log_estimate, upper_bound, lower_bound, interval

In [24]:
get_log_estimate(3, 20, next_to_river=True, high_confidence=False)

(2.7767581914803987, 2.964270326677529, 2.5892460562832684, 68)

In [25]:
np.median(boston_dataset.target)

21.2

In [26]:
# Challenge: Convert log price estimate using 1970s prices 
# as well as the upper and lower bounds to today's prices. 
# Round the values to the nearest 1000 dollars.

ZILLOW_MEDIAN_PRICE = 583.3    #price in thousands
SCALE_FACTOR = ZILLOW_MEDIAN_PRICE / np.median(boston_dataset.target)


log_est, upper, lower, conf = get_log_estimate(9, students_per_classroom=15,
                                               next_to_river=False, 
                                               high_confidence=False)


# Convert to today's dollars
dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
dollar_low = np.e**lower * 1000 * SCALE_FACTOR

# Round the dollar values to nearest thousand 
rounded_est = np.around(dollar_est, -3)
rounded_hi = np.around(dollar_hi, -3)
rounded_low = np.around(dollar_low, -3)


print(f'The estimated property value is {rounded_est}.')
print(f'At {conf}% confidence the valuation range is:')
print(f'USD {rounded_low} at the lower end to USD {rounded_hi} at the high end.')

The estimated property value is 827000.0.
At 68% confidence the valuation range is:
USD 685000.0 at the lower end to USD 997000.0 at the high end.


In [52]:
def get_dollar_estimate(rm, ptratio, chas=False, large_range=True):
    """Estimate the price of a property in Boston.
    
    Keyword Arguments:
    rm -- number of rooms in the property 
    ptratio -- number of students per teacher in classroom
    chas -- is house by river (True/False)
    large_range = 95%(True) or 68%(False) prediction interval
    """
    if rm < 1 or ptratio < 1: 
        print('That is unrealistic. Try again.')
        return

    log_est, upper, lower, conf = get_log_estimate(rm, students_per_classroom=ptratio,
                                                   next_to_river=chas, 
                                                   high_confidence=large_range)

    # Convert to today's dollars
    dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
    dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
    dollar_low = np.e**lower * 1000 * SCALE_FACTOR

    # Round the dollar values to nearest thousand 
    rounded_est = np.around(dollar_est, -3)
    rounded_hi = np.around(dollar_hi, -3)
    rounded_low = np.around(dollar_low, -3)

    print(f'The estimated property value is {rounded_est}.')
    print(f'At {conf}% confidence the valuation range is:')
    print(f'USD {rounded_low} at the lower end to USD {rounded_hi} at the high end.')

In [54]:
get_dollar_estimate(rm=1, ptratio=30, chas=True)

The estimated property value is 254000.0.
At 95% confidence the valuation range is:
USD 174000.0 at the lower end to USD 369000.0 at the high end.
